In [1]:
# Importing necessary packages
import pandas as pd
import numpy as np
import json
import nltk
from nltk.corpus import stopwords
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
# Uploading Dataset and inspecting its contents
df = pd.DataFrame()
df = pd.read_csv('/Users/dwreiter/Desktop/Work/Springboard/Video Game Capstone Project/Data Wrangling/Video_Games_Sales_as_at_22_Dec_2016.csv', index_col=0)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
Name,,,,,,,,,,,,,,,
Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Dropped all missing values
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6825 entries, Wii Sports to STORM: Frontline Nation
Data columns (total 15 columns):
Platform           6825 non-null object
Year_of_Release    6825 non-null float64
Genre              6825 non-null object
Publisher          6825 non-null object
NA_Sales           6825 non-null float64
EU_Sales           6825 non-null float64
JP_Sales           6825 non-null float64
Other_Sales        6825 non-null float64
Global_Sales       6825 non-null float64
Critic_Score       6825 non-null float64
Critic_Count       6825 non-null float64
User_Score         6825 non-null object
User_Count         6825 non-null float64
Developer          6825 non-null object
Rating             6825 non-null object
dtypes: float64(9), object(6)
memory usage: 853.1+ KB


In [4]:
# Changed Release Year to integers
df['Year_of_Release'] = (df['Year_of_Release']).astype(int)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
Name,,,,,,,,,,,,,,,
Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
New Super Mario Bros.,DS,2006,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
Wii Play,Wii,2006,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E


In [5]:
# Seeing how many types of genres there are
df['Genre'].value_counts()

Action          1630
Sports           943
Shooter          864
Role-Playing     712
Racing           581
Platform         403
Misc             384
Fighting         378
Simulation       297
Strategy         267
Adventure        248
Puzzle           118
Name: Genre, dtype: int64

In [6]:
# Making columns for each type of game
df['Action'] = (df['Genre'] == 'Action').astype(int)
df['Adventure'] = (df['Genre'] == 'Adventure').astype(int)
df['Fighter'] = (df['Genre'] == 'Fighting').astype(int)
df['Platformer'] = (df['Genre'] == 'Platform').astype(int)
df['Puzzler'] = (df['Genre'] == 'Puzzle').astype(int)
df['Racing'] = (df['Genre'] == 'Racing').astype(int)
df['RPG'] = (df['Genre'] == 'Role-Playing').astype(int)
df['Shooter'] = (df['Genre'] == 'Shooter').astype(int)
df['Simulation'] = (df['Genre'] == 'Simulation').astype(int)
df['Sports'] = (df['Genre'] == 'Sports').astype(int)
df['Strategy'] = (df['Genre'] == 'Strategy').astype(int)
df['Variety'] = (df['Genre'] == 'Misc').astype(int)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,...,Fighter,Platformer,Puzzler,Racing,RPG,Shooter,Simulation,Sports,Strategy,Variety
Name,,,,,,,,,,,,,,,,,,,,,
Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,...,0,0,0,0,0,0,0,1,0,0
Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,...,0,0,0,1,0,0,0,0,0,0
Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,...,0,0,0,0,0,0,0,1,0,0
New Super Mario Bros.,DS,2006,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,...,0,1,0,0,0,0,0,0,0,0
Wii Play,Wii,2006,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
# Seeing how many types of consoles there are
df['Platform'].value_counts()

PS2     1140
X360     858
PS3      769
PC       651
XB       565
Wii      479
DS       464
PSP      390
GC       348
PS4      239
GBA      237
XOne     159
3DS      155
PS       150
PSV      118
WiiU      89
DC        14
Name: Platform, dtype: int64

In [8]:
# Making columns for some of the consoles
df['Game Boy Advance'] = (df['Platform'] == 'GBA').astype(int)
df['Nintendo 3DS'] = (df['Platform'] == '3DS').astype(int)
df['Nintendo DS'] = (df['Platform'] == 'DS').astype(int)
df['Nintendo Gamecube'] = (df['Platform'] == 'GC').astype(int)
df['Nintendo Wii'] = (df['Platform'] == 'Wii').astype(int)
df['Nintendo Wii U'] = (df['Platform'] == 'WiiU').astype(int)
df['PC'] = (df['Platform'] == 'PC').astype(int)
df['Playstation'] = (df['Platform'] == 'PS').astype(int)
df['Playstation 2'] = (df['Platform'] == 'PS2').astype(int)
df['Playstation 3'] = (df['Platform'] == 'PS3').astype(int)
df['Playstation 4'] = (df['Platform'] == 'PS4').astype(int)
df['Playstation Vita'] = (df['Platform'] == 'PSV').astype(int)
df['PSP'] = (df['Platform'] == 'PSP').astype(int)
df['Sega Dreamcast'] = (df['Platform'] == 'DC').astype(int)
df['XBOX'] = (df['Platform'] == 'XB').astype(int)
df['XBOX 360'] = (df['Platform'] == 'X360').astype(int)
df['XBOX One'] = (df['Platform'] == 'XOne').astype(int)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,...,Playstation,Playstation 2,Playstation 3,Playstation 4,Playstation Vita,PSP,Sega Dreamcast,XBOX,XBOX 360,XBOX One
Name,,,,,,,,,,,,,,,,,,,,,
Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,...,0,0,0,0,0,0,0,0,0,0
Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,...,0,0,0,0,0,0,0,0,0,0
Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,...,0,0,0,0,0,0,0,0,0,0
New Super Mario Bros.,DS,2006,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,...,0,0,0,0,0,0,0,0,0,0
Wii Play,Wii,2006,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Saving DataFame to CSV
df.to_csv('vg_df.csv', sep='\t')

In [18]:
# Uploading second Dataset and inspecting its contents
df_2 = pd.read_csv('/Users/dwreiter/Desktop/Work/Springboard/Video Game Capstone Project/Data Wrangling/metacritic_game_user_comments.csv', index_col=0)
df_2.head()

,Title,Platform,Userscore,Comment,Username
0,The Legend of Zelda: Ocarina of Time,Nintendo64,10,"Everything in OoT is so near at perfection, it...",SirCaestus
1,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I won't bore you with what everyone is already...,Kaistlin
2,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Anyone who gives the masterpiece below a 7 or ...,Jacody
3,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I'm one of those people who think that this is...,doodlerman
4,The Legend of Zelda: Ocarina of Time,Nintendo64,10,This game is the highest rated game on Metacr...,StevenA


In [19]:
# Dropped all missing values
df_2 = df_2.dropna()
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283957 entries, 0 to 283982
Data columns (total 5 columns):
Title        283957 non-null object
Platform     283957 non-null object
Userscore    283957 non-null int64
Comment      283957 non-null object
Username     283957 non-null object
dtypes: int64(1), object(4)
memory usage: 13.0+ MB


In [37]:
df_2.iloc[14, 3]

'El juego perfecto. Insuperable por lo que supuso. No habr\xc3\xa1 nunca una revoluci\xc3\xb3n igual. Lo mejor de la \xc3\xa9poca y un espejo en el que miran los juegos de ahora. Seguir\xc3\xa1 en lo mas alto por siempre'

In [38]:
# Function to tokenize words from Amazon Reviews
def sent_tokenizer(x):
    """Tokenizes words, removes stops words, breaks down the stems and counts the tokens"""
    x = x.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    w_tokens = tokenizer.tokenize(x)
    sr = stopwords.words('english')
    for token in w_tokens:
        if token in sr:
            w_tokens.remove(token)
    ps = PorterStemmer()
    w_tokens2 = [ps.stem(w) for w in w_tokens]
    w_count = Counter(w_tokens2)
    return w_count, w_tokens2

# Creating a dictionary of dictionaries with words and their counts
tokens = df_2['Comment'][:14].apply(lambda x: sent_tokenizer(x)[0])
tokens

0     {u'perfect': 1, u'huge': 1, u'rate': 1, u'amon...
1     {u'tri': 1, u'on': 1, u'won': 1, u'fenc': 1, u...
2     {u'engag': 2, u'give': 2, u'is': 2, u'detract'...
3     {u'all': 5, u'consider': 2, u'abil': 1, u'stil...
4     {u'heart': 2, u'all': 2, u'almost': 2, u'in': ...
5     {u'and': 1, u'enjoy': 1, u'set': 2, u'creat': ...
6     {u'love': 1, u'it': 1, u'past': 1, u'second': ...
7     {u'all': 4, u'just': 1, u'show': 1, u'sage': 1...
8     {u'all': 3, u'think': 1, u'help': 2, u'sometim...
9     {u'enjoy': 2, u'and': 1, u'right': 1, u'favour...
10    {u'all': 1, u'often': 1, u'pro': 1, u'it': 2, ...
11    {u'unchart': 1, u'is': 2, u'video': 1, u'in': ...
12    {u'tombston': 1, u'cartridg': 1, u'is': 4, u'r...
13    {u'troll': 1, u'a': 1, u'all': 1, u'magic': 1,...
Name: Comment, dtype: object

In [39]:
# Making a list of the dictionary
tokens = list(tokens)

In [41]:
# Creating a DataFrame of the list
df_w = pd.DataFrame(tokens)
df_w.head()

,12,13,1998,1999,2,2007,2010,3,360,3d,...,write,year,yet,you,young,your,yourself,z,zelda,zone
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2.0,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN
4,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# Naming the index, changing missing values to 0
df_w.index.name = 'Reviews'
df_w = df_w.replace(pd.np.nan, 0)
df_w.head()

,12,13,1998,1999,2,2007,2010,3,360,3d,...,write,year,yet,you,young,your,yourself,z,zelda,zone
Reviews,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
4,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
# Putting DataFrame through a TF-IDF Transformer
transformer = TfidfTransformer()
matrix = transformer.fit_transform(df_w)
matrix = matrix.todense()
df_wt = pd.DataFrame(matrix, columns=df_w.columns)
df_wt.index.name = 'Reviews'
df_wt.head()

,12,13,1998,1999,2,2007,2010,3,360,3d,...,write,year,yet,you,young,your,yourself,z,zelda,zone
Reviews,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,...,0.0,0.000000,0.000000,0.139534,0.000000,0.0,0.0,0.0,0.000000,0.0
2,0.000000,0.0,0.059537,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,...,0.0,0.000000,0.000000,0.041762,0.000000,0.0,0.0,0.0,0.107629,0.0
3,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.046596,0.04033,...,0.0,0.000000,0.080659,0.050341,0.071767,0.0,0.0,0.0,0.000000,0.0
4,0.167262,0.0,0.000000,0.0,0.0,0.0,0.167262,0.0,0.000000,0.00000,...,0.0,0.144768,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0


In [ ]:
# Possibly for Merging
#df_wt = df_wt.reset_index()
#df_wt.groupby(df_wt['Comment'])

In [44]:
# Saving DataFame to CSV if needed
#df_wt.to_csv('vg_nlp_df.csv', sep='\t')

In [45]:
# Uploading third Dataset and inspecting its contents
df_3 = pd.DataFrame()
df_3 = pd.read_csv('/Users/dwreiter/Desktop/Work/Springboard/Video Game Capstone Project/Data Wrangling/Managerial_and_Decision_Economics_2013_Video_Games_Dataset.csv', index_col=0)
df_3.head()

,Title,US Sales (millions),Block4,Block2,Block1,Block0.5,YearReleased,2004,2005,2006,...,Tricks,Volleyball,Wakeboarding,Wrestling,FirstPerson,Platform,Isometric,SideScrolling,TopDown,ThirdPerson
Console,,,,,,,,,,,,,,,,,,,,,
Nintendo DS,Super Mario 64 DS,4.69,1,1,1,1,2004,1,0,0,...,0,0,0,0,0,1,0,0,0,1
Sony PSP,Lumines: Puzzle Fusion,0.56,0,0,0,1,2004,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Nintendo DS,WarioWare Touched!,0.54,0,0,0,1,2004,1,0,0,...,0,0,0,0,0,1,1,1,1,1
Sony PSP,Hot Shots Golf: Open Tee,0.49,0,0,0,0,2004,1,0,0,...,0,0,0,0,0,0,0,0,0,1
Nintendo DS,Spider-Man 2,0.45,0,0,0,0,2004,1,0,0,...,0,0,0,0,0,1,0,1,0,1


In [46]:
# No NaNs in this DataFrame
df_3.isnull().values.any()

False

In [47]:
# Saving DataFame to CSV
df_3.to_csv('vg_2_df.csv', sep='\t')